In [1]:
Pkg.update() 
Pkg.add("JuMP")
Pkg.add("GLPKMathProgInterface")
Pkg.add("Cpp")
Pkg.add("Cxx")

INFO: Updating METADATA...
INFO: Computing changes...
INFO: No packages to install, update or remove
INFO: Package JuMP is already installed
INFO: Package GLPKMathProgInterface is already installed
INFO: Package Cpp is already installed
INFO: Package Cxx is already installed


In [2]:
using JuMP
using MathProgBase
using GLPKMathProgInterface
using Cpp

In [9]:
include("util.jl")
include("network.jl")

In [10]:
read_nnet("small_nnet.txt")

Network(Layer[Layer([1.5; 1.5], [1.0; 1.0]), Layer([2.5; 2.5], [2.0 2.0; 2.0 2.0]), Layer([3.5], [3.0 3.0])])

In [11]:
read_nnet("ACASXU_nnet_1.txt")

Network(Layer[Layer([0.22763; -0.188762; … ; -0.44755; 0.175917], [0.0540062 0.0540062 … -0.180027 0.242194; -1.12374 -1.12374 … -0.00917929 0.055623; … ; -1.57631 -1.57631 … -0.00832176 -0.0926607; 0.620367 0.620367 … -0.0146141 0.262575]), Layer([0.08355; -0.0213261; … ; -0.426623; 0.780925], [-0.184202 -0.184202 … 0.337388 0.395671; 0.0171639 0.0171639 … -0.0371157 0.0196442; … ; -1.24508 -1.24508 … 0.646698 -0.159838; -0.348713 -0.348713 … -0.60238 -0.142155]), Layer([-0.408486; 1.06875; … ; -0.464336; 0.114195], [0.146062 0.146062 … -1.15657 0.515279; -1.159 -1.159 … -2.33211 -0.517898; … ; -1.14982 -1.14982 … -1.17371 0.414968; -0.684643 -0.684643 … -0.064733 -0.186417]), Layer([1.48416; -0.0392727; … ; 0.0762837; -0.216393], [-0.775044 -0.775044 … 0.00288769 -0.0587614; -0.289933 -0.289933 … 0.0289771 0.0943521; … ; 0.124486 0.124486 … -0.0140547 0.0703901; 0.0862763 0.0862763 … 0.0219731 -0.0403008]), Layer([0.343884; -0.148704; … ; -0.331041; -0.654165], [-0.169473 -0.169473 …

In [ ]:
function add_input_constraints(inputs::Matrix{Float64}, m::Model)
    n_inputs = len(network.layers[1,:])
    x_1 = @variable(m, x[1:n_inputs])
    for input in inputs # getting rows or columns?
        @constraint(m, x_1, x_i[i] == input[i])
    end
end

In [ ]:
function add_output_constraints(outputs::Matrix{Float64}, m::Model)
    n_layers = len(network.layers)
    n_outputs = len(network.layers[n_layers,:])
    x_k = @variable(m, x[1:n_outputs])
    for output in outputs
        @constraint(m, x_k, x_k[i] == input[i])
    end
end

In [ ]:
function add_network_constraints(network::Network, m::Model)
    for layer in network.layers # iterate by slice
        l = len(layer.biases)
        x_n = @variable(m, x[1:l])
        M = 1 # is this sufficent for ReLU constraint
        delta = @variable(m, x[1:l], Bin)
        @constraint(m, x_n, x[i] >= ((x_n * layer.weights) + layer.biases)[i])
        @constraint(m, x_n, x[i] <= ((x_n * layer.weights) + layer.biases)[i] + M*delta[i])
        @constraint(m, x_n, x[i] >= 0)
        @constraint(m, x_n, x[i] <= M*(1 - delta[i]))  
    end
end

In [ ]:
function initialize_constraints(inputs::Matrix{Float64}, outputs::Matrix{Float64},
                                network::Network, m::Model)
    add_input_constraints(inputs, m)
    #add_output_constraints(outputs, m)
    #add_network_constraints(network, m)
end

Make sure to clone the Reluplex repo to get the nnet reader. 

`https://github.com/guykatzz/ReluplexCav2017`

Then, cd into the nnet folder and run the following command to compile nnet into a dynamic library:

`g++ -shared -fPIC nnet.cpp -o nnet.so`

We will now import the dynamic library such that we can get the network layers from the .nnet files.

In [ ]:
const nnet_library_path =  pwd() * "/ReluplexCav2017/nnet/nnet.so"

In [ ]:
nnet_lib = Libdl.dlopen(nnet_library_path, Libdl.RTLD_GLOBAL)

In [ ]:
@cpp ccall((:load_network, nnet_lib), Float64, (Float64,), x)

In [ ]:
# Setup our model and add our contraints
model = Model(solver = GLPKSolverLP()) # use GLPK LP as our solver
initialize_constraints(inputs, outputs, network, model)

# Solve the model
status = solve(model)

# Print out the solution
print(status)